In [125]:
import sqlite3
import csv

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS Customers")

# # 테이블 생성 (CustomerID 자동 증가)
# cursor.execute("""
# CREATE TABLE Customers (
#     CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
#     CustomerName TEXT,
#     ContactName TEXT,
#     Address TEXT,
#     City TEXT,
#     PostalCode TEXT,
#     Country TEXT
# );
# """)

# # CSV 불러오기 (customerID 무시)
# with open("customers.csv", encoding="utf-8") as f:
#     reader = csv.DictReader(f)
#     for row in reader:
#         cursor.execute("""
#             INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
#             VALUES (?, ?, ?, ?, ?, ?)
#         """, (
#             row['companyName'],
#             row['contactName'],
#             row['address'],
#             row['city'],
#             row['postalCode'],
#             row['country']
#         ))


# w3schools.com 예제 실습에서는 CustomerID가 정수형으로 나오지만, 가져온 데이터에서는 CustomerID가 문자형으로 되어있어서 정수형으로 바꿔 진행
# 하지만 나중에 Customers 테이블과 Orders 테이블에 존재하는 CustomerID가 달라져 잘못된 예측 결과를 가져옴.
# 데이터 타입 불일치는 버그의 은상이자 성능 저하의 주범! 쿼리를 작성하기 전에 항상 조인하거나 비교할 열의 데이터 타입이 동일한지 확인하는 습관을 갖자.
# 따라서 Customers 테이블의 CustomerID를 다시 문자열로 바꿔 진행.

cursor.execute("""
CREATE TABLE Customers (
    CustomerID TEXT PRIMARY KEY,
    CustomerName TEXT,
    ContactName TEXT,
    Address TEXT,
    City TEXT,
    PostalCode TEXT,
    Country TEXT
);
""")

with open("customers.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Customers (CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            row['customerID'],
            row['companyName'],
            row['contactName'],
            row['address'],
            row['city'],
            row['postalCode'],
            row['country']
        ))
conn.commit()



In [126]:
cursor.execute("DROP TABLE IF EXISTS Products;")

cursor.execute("""
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    SupplierID INTEGER,
    CategoryID INTEGER,
    QuantityPerUnit TEXT,
    UnitPrice REAL,
    UnitsInStock INTEGER,
    UnitsOnOrder INTEGER,
    ReorderLevel INTEGER,
    Discontinued INTEGER
);
""")

with open("products.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Products (
                ProductID, ProductName, SupplierID, CategoryID,
                QuantityPerUnit, UnitPrice, UnitsInStock,
                UnitsOnOrder, ReorderLevel, Discontinued
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['productID']),
            row['productName'],
            int(row['supplierID']),
            int(row['categoryID']),
            row['quantityPerUnit'],
            float(row['unitPrice']),
            int(row['unitsInStock']),
            int(row['unitsOnOrder']),
            int(row['reorderLevel']),
            int(row['discontinued'])
        ))
conn.commit()


In [127]:
cursor.execute("DROP TABLE IF EXISTS Order_details")

# 새 테이블 생성
cursor.execute("""
CREATE TABLE Order_details (
    orderID INTEGER,
    productID INTEGER,
    unitPrice REAL,
    quantity INTEGER,
    discount REAL
)
""")

with open('order_details.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Order_details (
                orderID, productID,
                unitPrice, quantity, discount
            )
            VALUES (?, ?, ?, ?, ?)
        """, (
            int(row['orderID']),
            int(row['productID']),
            float(row['unitPrice']),
            int(row['quantity']),
            float(row['discount'])
        ))

conn.commit()

In [128]:
cursor.execute("DROP TABLE IF EXISTS Orders")

cursor.execute("""
CREATE TABLE Orders (
    orderID INTEGER,
    customerID TEXT,
    employeeID INTEGER,
    orderDate TEXT,
    requiredDate TEXT,
    shippedDate TEXT,
    shipVia INTEGER,
    freight REAL,
    shipName TEXT,
    shipAddress TEXT,
    shipCity TEXT,
    shipRegion TEXT,
    shipPostalCode TEXT,
    shipCountry TEXT
)
""")

with open('orders.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Orders (
                orderID, customerID, employeeID,
                orderDate, requiredDate, shippedDate,
                shipVia, freight, shipName,
                shipAddress, shipCity, shipRegion,
                shipPostalCode, shipCountry
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['orderID']),
            row['customerID'],
            int(row['employeeID']),
            row['orderDate'],
            row['requiredDate'],
            row['shippedDate'],
            int(row['shipVia']),
            float(row['freight']),
            row['shipName'],
            row['shipAddress'],
            row['shipCity'],
            row['shipRegion'] if row['shipRegion'] != 'NULL' else None,
            row['shipPostalCode'],
            row['shipCountry']
        ))

conn.commit()

In [162]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Categories (
    CategoryID INTEGER PRIMARY KEY,
    CategoryName TEXT,
    Description TEXT,
    Picture BLOB
);
""")

with open("categories.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Categories (CategoryID, CategoryName, Description, Picture)
            VALUES (?, ?, ?, ?)
        """, (
            int(row['categoryID']),
            row['categoryName'],
            row['description'],
            bytes.fromhex(row['picture'][2:]) if row['picture'].startswith("0x") else row['picture']
        ))
conn.commit()

In [170]:
cursor.execute("DROP TABLE IF EXISTS Employees;")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Employees (
    EmployeeID INTEGER PRIMARY KEY,
    LastName TEXT,
    FirstName TEXT,
    Title TEXT,
    TitleOfCourtesy TEXT,
    BirthDate TEXT,
    HireDate TEXT,
    Address TEXT,
    City TEXT,
    Region TEXT,
    PostalCode TEXT,
    Country TEXT,
    HomePhone TEXT,
    Extension TEXT,
    Photo BLOB,
    Notes TEXT,
    ReportsTo INTEGER,
    PhotoPath TEXT
);
""")

def parse_blob(value):
    if value.startswith("0x"):
        return bytes.fromhex(value[2:])
    return value

def parse_nullable_int(value):
    return int(value) if value and value.upper() != 'NULL' else None

with open("employees.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Employees (
                EmployeeID, LastName, FirstName, Title, TitleOfCourtesy,
                BirthDate, HireDate, Address, City, Region, PostalCode,
                Country, HomePhone, Extension, Photo, Notes, ReportsTo, PhotoPath
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['employeeID']),
            row['lastName'],
            row['firstName'],
            row['title'],
            row['titleOfCourtesy'],
            row['birthDate'],
            row['hireDate'],
            row['address'],
            row['city'],
            row['region'],
            row['postalCode'],
            row['country'],
            row['homePhone'],
            row['extension'],
            parse_blob(row['photo']),
            row['notes'],
            parse_nullable_int(row['reportsTo']),
            row['photoPath']
        ))
conn.commit()


In [175]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Suppliers (
    SupplierID INTEGER PRIMARY KEY,
    CompanyName TEXT,
    ContactName TEXT,
    ContactTitle TEXT,
    Address TEXT,
    City TEXT,
    Region TEXT,
    PostalCode TEXT,
    Country TEXT,
    Phone TEXT,
    Fax TEXT,
    HomePage TEXT
);
""")

def clean_nullable(val):
    return None if val.upper() == 'NULL' or val.strip() == '' else val

with open("suppliers.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cursor.execute("""
            INSERT INTO Suppliers (
                SupplierID, CompanyName, ContactName, ContactTitle,
                Address, City, Region, PostalCode, Country,
                Phone, Fax, HomePage
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['supplierID']),
            row['companyName'],
            row['contactName'],
            row['contactTitle'],
            row['address'],
            row['city'],
            clean_nullable(row['region']),
            row['postalCode'],
            row['country'],
            row['phone'],
            clean_nullable(row['fax']),
            clean_nullable(row['homePage'])
        ))
conn.commit()


In [129]:
cursor.execute("SELECT * FROM Customers;")
for row in cursor.fetchall():
    print(row)

('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen  8', 'Luleå', 'S-958 22', 'Sweden')
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('BLONP', 'Blondesddsl père et fils', 'Frédérique Citeaux', '24 place Kléber', 'Strasbourg', '67000', 'France')
('BOLID', 'Bólido Comidas preparadas', 'Martín Sommer', '67C Araquil', 'Madrid', '28023', 'Spain')
('BONAP', "Bon app'", 'Laurence Lebihan', '12 rue des Bouchers', 'Marseille', '13008', 'France')
('BOTTM', 'Bottom-Dollar Markets', 'Elizabeth Lincoln', '23 T

In [130]:
# 예제 실행 함수
def run_query(title, query):
    print(f"\n {title}")
    print(f"SQL : {query}")
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

In [131]:
run_query("SQL Syntax", "SELECT * FROM Customers;")


 SQL Syntax
SQL : SELECT * FROM Customers;
('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen  8', 'Luleå', 'S-958 22', 'Sweden')
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('BLONP', 'Blondesddsl père et fils', 'Frédérique Citeaux', '24 place Kléber', 'Strasbourg', '67000', 'France')
('BOLID', 'Bólido Comidas preparadas', 'Martín Sommer', '67C Araquil', 'Madrid', '28023', 'Spain')
('BONAP', "Bon app'", 'Laurence Lebihan', '12 rue des Bouchers', 'Marseille', '13008', 'France')
('BOTTM', 'Bottom

In [132]:
run_query("SQL SELECT", "SELECT CustomerName,City FROM Customers;")


 SQL SELECT
SQL : SELECT CustomerName,City FROM Customers;
('Alfreds Futterkiste', 'Berlin')
('Ana Trujillo Emparedados y helados', 'México D.F.')
('Antonio Moreno Taquería', 'México D.F.')
('Around the Horn', 'London')
('Berglunds snabbköp', 'Luleå')
('Blauer See Delikatessen', 'Mannheim')
('Blondesddsl père et fils', 'Strasbourg')
('Bólido Comidas preparadas', 'Madrid')
("Bon app'", 'Marseille')
('Bottom-Dollar Markets', 'Tsawassen')
("B's Beverages", 'London')
('Cactus Comidas para llevar', 'Buenos Aires')
('Centro comercial Moctezuma', 'México D.F.')
('Chop-suey Chinese', 'Bern')
('Comércio Mineiro', 'Sao Paulo')
('Consolidated Holdings', 'London')
('Drachenblut Delikatessen', 'Aachen')
('Du monde entier', 'Nantes')
('Eastern Connection', 'London')
('Ernst Handel', 'Graz')
('Familia Arquibaldo', 'Sao Paulo')
('FISSA Fabrica Inter. Salchichas S.A.', 'Madrid')
('Folies gourmandes', 'Lille')
('Folk och fä HB', 'Bräcke')
('Frankenversand', 'München')
('France restauration', 'Nantes')


In [133]:
run_query("SQL Select Distinct", "SELECT DISTINCT Country FROM Customers;")

run_query("SQL Select Distinct / Count Distinct", "SELECT COUNT(DISTINCT Country) FROM Customers;")

run_query("SQL Select Distinct / Count Distinct 2", "SELECT Count(*) AS DistinctCountries FROM (SELECT DISTINCT Country FROM Customers);")


 SQL Select Distinct
SQL : SELECT DISTINCT Country FROM Customers;
('Germany',)
('Mexico',)
('UK',)
('Sweden',)
('France',)
('Spain',)
('Canada',)
('Argentina',)
('Switzerland',)
('Brazil',)
('Austria',)
('Italy',)
('Portugal',)
('USA',)
('Venezuela',)
('Ireland',)
('Belgium',)
('Norway',)
('Denmark',)
('Finland',)
('Poland',)

 SQL Select Distinct / Count Distinct
SQL : SELECT COUNT(DISTINCT Country) FROM Customers;
(21,)

 SQL Select Distinct / Count Distinct 2
SQL : SELECT Count(*) AS DistinctCountries FROM (SELECT DISTINCT Country FROM Customers);
(21,)


In [134]:
run_query("SQL Where", "SELECT * FROM Customers WHERE Country='Mexico';")

run_query("Sql Where / Text Fields vs Numeric Fields", "SELECT * FROM Customers WHERE CustomerID=1;")

run_query("SQL Where / Operators in the WHERE Clause", "SELECT * FROM Customers WHERE CustomerID > 80;")


 SQL Where
SQL : SELECT * FROM Customers WHERE Country='Mexico';
('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('CENTC', 'Centro comercial Moctezuma', 'Francisco Chang', 'Sierras de Granada 9993', 'México D.F.', '05022', 'Mexico')
('PERIC', 'Pericles Comidas clásicas', 'Guillermo Fernández', 'Calle Dr. Jorge Cash 321', 'México D.F.', '05033', 'Mexico')
('TORTU', 'Tortuga Restaurante', 'Miguel Angel Paolino', 'Avda. Azteca 123', 'México D.F.', '05033', 'Mexico')

 Sql Where / Text Fields vs Numeric Fields
SQL : SELECT * FROM Customers WHERE CustomerID=1;

 SQL Where / Operators in the WHERE Clause
SQL : SELECT * FROM Customers WHERE CustomerID > 80;
('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana 

In [135]:
run_query("SQL Order By", "SELECT * FROM Products ORDER BY unitPrice;")

run_query("SQL Order By / DESC", "SELECT * FROM Products ORDER BY unitPrice DESC;")

run_query("SQL Order By / Order Alphabetically", "SELECT * FROM Products ORDER BY ProductName;")

run_query("SQL Order By / Alphabetically DESC", "SELECT * FROM Products ORDER BY ProductName DESC;")

run_query("SQL Order By / ORDER BY Several Columns", "SELECT * FROM Customers ORDER BY Country, CustomerName;")

run_query("SQL Order By / Using Both ASC and DESC", "SELECT * FROM Customers ORDER BY Country ASC, customerName DESC;")


 SQL Order By
SQL : SELECT * FROM Products ORDER BY unitPrice;
(33, 'Geitost', 15, 4, '500 g', 2.5, 112, 0, 20, 0)
(24, 'Guaraná Fantástica', 10, 1, '12 - 355 ml cans', 4.5, 20, 0, 0, 1)
(13, 'Konbu', 6, 8, '2 kg box', 6.0, 24, 0, 5, 0)
(52, 'Filo Mix', 24, 5, '16 - 2 kg boxes', 7.0, 38, 0, 25, 0)
(54, 'Tourtière', 25, 6, '16 pies', 7.45, 21, 0, 10, 0)
(75, 'Rhönbräu Klosterbier', 12, 1, '24 - 0.5 l bottles', 7.75, 125, 0, 25, 0)
(23, 'Tunnbröd', 9, 5, '12 - 250 g pkgs.', 9.0, 61, 0, 25, 0)
(19, 'Teatime Chocolate Biscuits', 8, 3, '10 boxes x 12 pieces', 9.2, 25, 0, 5, 0)
(45, 'Rogede sild', 21, 8, '1k pkg.', 9.5, 5, 70, 15, 0)
(47, 'Zaanse koeken', 22, 3, '10 - 4 oz boxes', 9.5, 36, 0, 0, 0)
(41, "Jack's New England Clam Chowder", 19, 8, '12 - 12 oz cans', 9.65, 85, 0, 10, 0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0, 13, 70, 25, 0)
(21, "Sir Rodney's Scones", 8, 3, '24 pkgs. x 4 pieces', 10.0, 3, 40, 5, 0)
(74, 'Longlife Tofu', 4, 7, '5 kg pkg.', 10.0, 4, 20, 5, 0)
(46,

In [136]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("현재 테이블 목록:")
for row in cursor.fetchall():
    print(" -", row[0])


현재 테이블 목록:
 - Customers
 - Products
 - Order_details
 - Orders


In [137]:
run_query("SQL And", "SELECT * FROM Customers WHERE Country = 'Spain' AND CustomerName LIKE 'G%';")

run_query("SQL And / All conditions Must Be True", "SELECT * FROM Customers WHERE Country = 'Brazil' AND City = 'Rio de Janeiro' AND CustomerID > 50;")

run_query("SQL And / Combining AND and OR", "SELECT * FROM Customers WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');")

run_query("SQL And / Combining AND and OR 2", "SELECT * FROM Customers WHERE Country = 'Spain' AND CustomerName LIKE 'G%' OR CustomerName LIKE 'R%';")


 SQL And
SQL : SELECT * FROM Customers WHERE Country = 'Spain' AND CustomerName LIKE 'G%';
('GALED', 'Galería del gastrónomo', 'Eduardo Saavedra', 'Rambla de Cataluña 23', 'Barcelona', '08022', 'Spain')
('GODOS', 'Godos Cocina Típica', 'José Pedro Freyre', 'C/ Romero 33', 'Sevilla', '41101', 'Spain')

 SQL And / All conditions Must Be True
SQL : SELECT * FROM Customers WHERE Country = 'Brazil' AND City = 'Rio de Janeiro' AND CustomerID > 50;
('HANAR', 'Hanari Carnes', 'Mario Pontes', 'Rua do Paço 67', 'Rio de Janeiro', '05454-876', 'Brazil')
('RICAR', 'Ricardo Adocicados', 'Janete Limeira', 'Av. Copacabana 267', 'Rio de Janeiro', '02389-890', 'Brazil')

 SQL And / Combining AND and OR
SQL : SELECT * FROM Customers WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
('GALED', 'Galería del gastrónomo', 'Eduardo Saavedra', 'Rambla de Cataluña 23', 'Barcelona', '08022', 'Spain')
('GODOS', 'Godos Cocina Típica', 'José Pedro Freyre', 'C/ Romero 33', 'Sevilla', '4

In [138]:
run_query("SQL Or", "SELECT * FROM Customers WHERE Country = 'Germany' OR Country = 'Spain';")

run_query("SQL Or / Combining AND and OR", "SELECT * FROM Customers WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');")

run_query("SQL Or / Combining AND and OR", "SELECT * FROM Customers WHERE Country = 'Spain' AND CustomerName LIKE 'G%' OR CustomerName LIKE 'R%';")


 SQL Or
SQL : SELECT * FROM Customers WHERE Country = 'Germany' OR Country = 'Spain';
('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('BOLID', 'Bólido Comidas preparadas', 'Martín Sommer', '67C Araquil', 'Madrid', '28023', 'Spain')
('DRACD', 'Drachenblut Delikatessen', 'Sven Ottlieb', 'Walserweg 21', 'Aachen', '52066', 'Germany')
('FISSA', 'FISSA Fabrica Inter. Salchichas S.A.', 'Diego Roel', 'C/ Moralzarzal 86', 'Madrid', '28034', 'Spain')
('FRANK', 'Frankenversand', 'Peter Franken', 'Berliner Platz 43', 'München', '80805', 'Germany')
('GALED', 'Galería del gastrónomo', 'Eduardo Saavedra', 'Rambla de Cataluña 23', 'Barcelona', '08022', 'Spain')
('GODOS', 'Godos Cocina Típica', 'José Pedro Freyre', 'C/ Romero 33', 'Sevilla', '41101', 'Spain')
('KOENE', 'Königlich Essen', 'Philip Cramer', 'Maubelstr. 90', 'Brandenburg', '14776', 'Germany'

In [139]:
run_query("SQL Not", "SELECT * FROM Customers WHERE NOT Country = 'Spain';")

run_query("SQL Not / NOT LIKE", "SELECT * FROM Customers WHERE CustomerName NOT LIKE 'A%';")

run_query("SQL Not / NOT BETWEEN", "SELECT * FROM Customers WHERE CustomerID NOT BETWEEN 10 AND 60;")

run_query("SQL Not / NOT IN", "SELECT * FROM Customers WHERE City NOT IN ('Paris', 'London');")

run_query("SQL Not / NOT Greater Than", "SELECT * FROM Customers WHERE NOT CustomerID > 50;")

run_query("SQL Not / NOT Less Than", "SELECT * FROM Customers WHERE NOT CustomerID < 50;")


 SQL Not
SQL : SELECT * FROM Customers WHERE NOT Country = 'Spain';
('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen  8', 'Luleå', 'S-958 22', 'Sweden')
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('BLONP', 'Blondesddsl père et fils', 'Frédérique Citeaux', '24 place Kléber', 'Strasbourg', '67000', 'France')
('BONAP', "Bon app'", 'Laurence Lebihan', '12 rue des Bouchers', 'Marseille', '13008', 'France')
('BOTTM', 'Bottom-Dollar Markets', 'Elizabeth Lincoln', '23 Tsawassen Blvd.', 'Tsawassen', 

In [140]:
run_query("SQL Insert Into", "INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country) VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');")

run_query("SQL Insert Into / Insert Data Only in Specified Columns", "INSERT INTO Customers (CustomerName, City, Country) VALUES ('Cardinal', 'Stavanger', 'Norway');")

run_query("SQL Insert Into / Insert Multiple Rows", "INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country) VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'), ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'), ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');")



 SQL Insert Into
SQL : INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country) VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');

 SQL Insert Into / Insert Data Only in Specified Columns
SQL : INSERT INTO Customers (CustomerName, City, Country) VALUES ('Cardinal', 'Stavanger', 'Norway');

 SQL Insert Into / Insert Multiple Rows
SQL : INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country) VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'), ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'), ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');


In [141]:
run_query("SQL Null Values / The IS NULL Operator", "SELECT CustomerName, ContactName, Address FROM Customers WHERE Address IS NULL;")

run_query("SQL Null Values / The IS NOT NULL Operator", "SELECT CustomerName, ContactName, Address FROM Customers WHERE Address IS NOT NULL;")


 SQL Null Values / The IS NULL Operator
SQL : SELECT CustomerName, ContactName, Address FROM Customers WHERE Address IS NULL;
('Cardinal', None, None)

 SQL Null Values / The IS NOT NULL Operator
SQL : SELECT CustomerName, ContactName, Address FROM Customers WHERE Address IS NOT NULL;
('Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57')
('Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222')
('Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos  2312')
('Around the Horn', 'Thomas Hardy', '120 Hanover Sq.')
('Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen  8')
('Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57')
('Blondesddsl père et fils', 'Frédérique Citeaux', '24 place Kléber')
('Bólido Comidas preparadas', 'Martín Sommer', '67C Araquil')
("Bon app'", 'Laurence Lebihan', '12 rue des Bouchers')
('Bottom-Dollar Markets', 'Elizabeth Lincoln', '23 Tsawassen Blvd.')
("B's Beverages", 'Victoria Ashworth', 'Fauntleroy Circus')
('Cactu

In [142]:
run_query("SQL Update", "UPDATE Customers SET ContactName = 'Alfred Schmidt', City = 'Frankfurt' WHERE CustomerID = 1;")

run_query("SQL Update / UPDATE Multiple Records", "UPDATE Customers SET ContactName = 'Juan' WHERE Country = 'Mexico';")

# run_query("SQL Update / Update Warning!", "UPDATE Customers SET ContactName = 'Juan';")


 SQL Update
SQL : UPDATE Customers SET ContactName = 'Alfred Schmidt', City = 'Frankfurt' WHERE CustomerID = 1;

 SQL Update / UPDATE Multiple Records
SQL : UPDATE Customers SET ContactName = 'Juan' WHERE Country = 'Mexico';


In [143]:
run_query("SQL Delete", "DELETE FROM Customers WHERE CustomerName = 'Alfreds Futterkiste';")

# run_query("SQL Delete / Delete All Records", "DELETE FROM Customers;")

# run_query("SQL Delete / Delete a Table", "DROP TABLE Customers;")


 SQL Delete
SQL : DELETE FROM Customers WHERE CustomerName = 'Alfreds Futterkiste';


In [144]:
# run_query("SQL Select Top", "SELECT TOP 3 * FROM Customers;") SQLite에서는 SELECT TOP을 지원하지 않아서 'LIMIT'절을 사용해야 된다.

run_query("SQL Select Top / LIMIT", "SELECT * FROM Customers LIMIT 3;")

# run_query("SQL Select Top / FETCH FIRST", "SELECT * FROM Customers FETCH FIRST 3 ROWS ONLY;") SQLite에서 FIRST 함수도 지원하지 않는다.

# run_query("SQL Select Top / SQL TOP PERCENT", "SELECT * FROM Customers FETCH FIRST 50 PERCENT ROWS ONLY;")

run_query("SQL Select Top / ADD a WHERE CLAUSE", "SELECT * FROM Customers WHERE Country = 'Germany' LIMIT 3;")

# run_query("SQL Select Top / ADD a WHERE CLAUSE", "SELECT * FROM Customers WHERE Country = 'Germany' FETCH FIRST 3 ROWS ONLY;")

run_query("SQL Select Top / ADD the ORDER BY Keyword", "SELECT * FROM Customers ORDER BY CustomerName DESC LIMIT 3;")

# run_query("SQL Select Top / ADD the ORDER BY Keyword", "SELECT * FROM Customers ORDER BY CustomerName DESC FETCH FIRST 3 ROWS ONLY;")


 SQL Select Top / LIMIT
SQL : SELECT * FROM Customers LIMIT 3;
('ANATR', 'Ana Trujillo Emparedados y helados', 'Juan', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Juan', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')

 SQL Select Top / ADD a WHERE CLAUSE
SQL : SELECT * FROM Customers WHERE Country = 'Germany' LIMIT 3;
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('DRACD', 'Drachenblut Delikatessen', 'Sven Ottlieb', 'Walserweg 21', 'Aachen', '52066', 'Germany')
('FRANK', 'Frankenversand', 'Peter Franken', 'Berliner Platz 43', 'München', '80805', 'Germany')

 SQL Select Top / ADD the ORDER BY Keyword
SQL : SELECT * FROM Customers ORDER BY CustomerName DESC LIMIT 3;
('WOLZA', 'Wolski  Zajazd', 'Zbyszek Piestrzeniewicz', 'ul. Filtrowa 68', 'Warszawa', '01-012', 'Poland')
('WILMK

In [145]:
run_query("SQL Min and Max", "SELECT MIN(unitPrice) FROM Products;")

run_query("SQL Min and Max", "SELECT MAX(unitPrice) FROM Products;")

run_query("SQL Min and Max / Set Column Name (Alias)", "SELECT MIN(unitPrice) AS SmallestPrice FROM Products;")

run_query("SQL Min and Max / Use MIN() with GROUP BY", "SELECT MIN(unitPrice) AS SmallestPrice, CategoryID FROM Products GROUP BY CategoryID;")




 SQL Min and Max
SQL : SELECT MIN(unitPrice) FROM Products;
(2.5,)

 SQL Min and Max
SQL : SELECT MAX(unitPrice) FROM Products;
(263.5,)

 SQL Min and Max / Set Column Name (Alias)
SQL : SELECT MIN(unitPrice) AS SmallestPrice FROM Products;
(2.5,)

 SQL Min and Max / Use MIN() with GROUP BY
SQL : SELECT MIN(unitPrice) AS SmallestPrice, CategoryID FROM Products GROUP BY CategoryID;
(4.5, 1)
(10.0, 2)
(9.2, 3)
(2.5, 4)
(7.0, 5)
(7.45, 6)
(10.0, 7)
(6.0, 8)


In [146]:
run_query("SQL Count", "SELECT COUNT(*) FROM Products;")

run_query("SQL Count / Specify Column", "SELECT COUNT(ProductName) FROM Products;")

run_query("SQL Count / Add a WHERE Clause", "SELECT COUNT(ProductID) FROM Products WHERE unitPrice > 20;")

run_query("SQL Count / Ignore Duplicates", "SELECT COUNT(DISTINCT unitPrice) FROM Products;")

run_query("SQL Count / Use and Alias", "SELECT COUNT(*) AS [Number of records] FROM Products;")

run_query("SQL Count / use COUNT() with GROUP BY", "SELECT COUNT(*) AS [Number of records], CategoryID FROM Products GROUP BY CategoryID;")


 SQL Count
SQL : SELECT COUNT(*) FROM Products;
(77,)

 SQL Count / Specify Column
SQL : SELECT COUNT(ProductName) FROM Products;
(77,)

 SQL Count / Add a WHERE Clause
SQL : SELECT COUNT(ProductID) FROM Products WHERE unitPrice > 20;
(37,)

 SQL Count / Ignore Duplicates
SQL : SELECT COUNT(DISTINCT unitPrice) FROM Products;
(62,)

 SQL Count / Use and Alias
SQL : SELECT COUNT(*) AS [Number of records] FROM Products;
(77,)

 SQL Count / use COUNT() with GROUP BY
SQL : SELECT COUNT(*) AS [Number of records], CategoryID FROM Products GROUP BY CategoryID;
(12, 1)
(12, 2)
(13, 3)
(10, 4)
(7, 5)
(6, 6)
(5, 7)
(12, 8)


In [147]:
run_query("SQL Sum", "SELECT SUM(Quantity) FROM Order_details;")

run_query("SQL Sum / Add a WHERE Clause", "SELECT SUM(Quantity) FROM Order_details WHERE ProductID = 11;")

run_query("SQL Sum / Use an Alias", "SELECT SUM(Quantity) AS total FROM Order_details;")

run_query("SQL Sum / Use SUM() with GROUP BY", "SELECT OrderID, SUM(Quantity) AS [Total Quantity] FROM Order_details GROUP BY OrderID;")

run_query("SQL Sum / SUM() with an Expression", "SELECT SUM(Quantity * 10) FROM Order_details;")


 SQL Sum
SQL : SELECT SUM(Quantity) FROM Order_details;
(51317,)

 SQL Sum / Add a WHERE Clause
SQL : SELECT SUM(Quantity) FROM Order_details WHERE ProductID = 11;
(706,)

 SQL Sum / Use an Alias
SQL : SELECT SUM(Quantity) AS total FROM Order_details;
(51317,)

 SQL Sum / Use SUM() with GROUP BY
SQL : SELECT OrderID, SUM(Quantity) AS [Total Quantity] FROM Order_details GROUP BY OrderID;
(10248, 27)
(10249, 49)
(10250, 60)
(10251, 41)
(10252, 105)
(10253, 102)
(10254, 57)
(10255, 110)
(10256, 27)
(10257, 46)
(10258, 121)
(10259, 11)
(10260, 102)
(10261, 40)
(10262, 29)
(10263, 184)
(10264, 60)
(10265, 50)
(10266, 12)
(10267, 135)
(10268, 14)
(10269, 80)
(10270, 55)
(10271, 24)
(10272, 70)
(10273, 152)
(10274, 27)
(10275, 18)
(10276, 25)
(10277, 32)
(10278, 64)
(10279, 15)
(10280, 62)
(10281, 11)
(10282, 8)
(10283, 76)
(10284, 61)
(10285, 121)
(10286, 140)
(10287, 75)
(10288, 13)
(10289, 39)
(10290, 60)
(10291, 46)
(10292, 20)
(10293, 33)
(10294, 75)
(10295, 4)
(10296, 57)
(10297, 80)
(

In [148]:
run_query("SQL Avg", "SELECT AVG(unitPrice) FROM Products;")

run_query("SQL Avg / Add a WHERE Clause", "SELECT AVG(unitPrice) FROM Products WHERE CategoryID = 1;")

run_query("SQL Avg / Use an Alias", "SELECT AVG(unitPrice) AS [average price] FROM Products;")

run_query("SQL Avg / Higher than Average", "SELECT * FROM Products WHERE unitprice > (SELECT AVG(unitprice) FROM Products);")

run_query("SQL Avg / Use AVG() with GROUP BY", "SELECT AVG(unitPrice) AS AveragePrice, CategoryID FROM Products GROUP BY CategoryID;")


 SQL Avg
SQL : SELECT AVG(unitPrice) FROM Products;
(28.866363636363637,)

 SQL Avg / Add a WHERE Clause
SQL : SELECT AVG(unitPrice) FROM Products WHERE CategoryID = 1;
(37.979166666666664,)

 SQL Avg / Use an Alias
SQL : SELECT AVG(unitPrice) AS [average price] FROM Products;
(28.866363636363637,)

 SQL Avg / Higher than Average
SQL : SELECT * FROM Products WHERE unitprice > (SELECT AVG(unitprice) FROM Products);
(7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0, 15, 0, 10, 0)
(8, 'Northwoods Cranberry Sauce', 3, 2, '12 - 12 oz jars', 40.0, 6, 0, 0, 0)
(9, 'Mishi Kobe Niku', 4, 6, '18 - 500 g pkgs.', 97.0, 29, 0, 0, 1)
(10, 'Ikura', 4, 8, '12 - 200 ml jars', 31.0, 31, 0, 0, 0)
(12, 'Queso Manchego La Pastora', 5, 4, '10 - 500 g pkgs.', 38.0, 86, 0, 0, 0)
(17, 'Alice Mutton', 7, 6, '20 - 1 kg tins', 39.0, 0, 0, 0, 1)
(18, 'Carnarvon Tigers', 7, 8, '16 kg pkg.', 62.5, 42, 0, 0, 0)
(20, "Sir Rodney's Marmalade", 8, 3, '30 gift boxes', 81.0, 40, 0, 0, 0)
(26, 'Gumbär G

In [149]:
run_query("SQL Like", "SELECT * FROM Customers WHERE CustomerName LIKE 'a%';")

run_query("SQL Like / The _ Wildcard", "SELECT * FROM Customers WHERE city LIKE 'L_nd__';")

run_query("SQL Like / The % Wildcard", "SELECT * FROM Customers WHERE City LIKE '%L%';")

run_query("SQL Like / Starts With", "SELECT * FROM Customers WHERE CustomerName LIKE 'La%';")

run_query("SQL Like / Ends With", "SELECT * FROM Customers WHERE CustomerName LIKE '%a';")

run_query("SQL Like / Starts With, Ends With", "SELECT * FROM Customers WHERE CustomerName LIKE 'b%s';")

run_query("SQL Like / Contains", "SELECT * FROM Customers WHERE CustomerName LIKE '%or%';")

run_query("SQL Like / Combine Wildcards", "SELECT * FROM Customers WHERE CustomerName LIKE 'a__%';")

run_query("SQL Like / Without Wildcard", "SELECT * FROM Customers WHERE Country LIKE 'Spain';")


 SQL Like
SQL : SELECT * FROM Customers WHERE CustomerName LIKE 'a%';
('ANATR', 'Ana Trujillo Emparedados y helados', 'Juan', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Juan', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')

 SQL Like / The _ Wildcard
SQL : SELECT * FROM Customers WHERE city LIKE 'L_nd__';
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
('BSBEV', "B's Beverages", 'Victoria Ashworth', 'Fauntleroy Circus', 'London', 'EC2 5NT', 'UK')
('CONSH', 'Consolidated Holdings', 'Elizabeth Brown', 'Berkeley Gardens 12  Brewery', 'London', 'WX1 6LT', 'UK')
('EASTC', 'Eastern Connection', 'Ann Devon', '35 King George', 'London', 'WX3 6FW', 'UK')
('NORTS', 'North/South', 'Simon Crowther', 'South House 300 Queensbridge', 'London', 'SW7 1RZ', 'UK')
('SEVES', 'Seven Seas Imports', 'Hari Kum

In [150]:
run_query("SQL Wildcards", "SELECT * FROM Customers WHERE CustomerName LIKE 'a%';")

run_query("SQL Wildcards / Using the % Wildcard", "SELECT * FROM Customers WHERE CustomerName LIKE '%es';")

run_query("SQL Wildcards / Using the _ Wildcard", "SELECT * FROM Customers WHERE City LIKE '_ondon';")

run_query("SQL Wildcards / Using the [] Wildcard", "SELECT * FROM Customers WHERE CustomerName LIKE '[bsp]%';")

run_query("SQL Wildcards / Using the - Wildcard", "SELECT * FROM Customers WHERE CustomerName LIKE '[a-f]%';")

run_query("SQL Wildcards / Combine Wildcards", "SELECT * FROM Customers WHERE CustomerName LIKE 'a__%';")

run_query("SQL Wildcards / Without Wildcard", "SELECT * FROM Customers WHERE Country LIKE 'Spain';")


 SQL Wildcards
SQL : SELECT * FROM Customers WHERE CustomerName LIKE 'a%';
('ANATR', 'Ana Trujillo Emparedados y helados', 'Juan', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
('ANTON', 'Antonio Moreno Taquería', 'Juan', 'Mataderos  2312', 'México D.F.', '05023', 'Mexico')
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')

 SQL Wildcards / Using the % Wildcard
SQL : SELECT * FROM Customers WHERE CustomerName LIKE '%es';
('BSBEV', "B's Beverages", 'Victoria Ashworth', 'Fauntleroy Circus', 'London', 'EC2 5NT', 'UK')
('FOLIG', 'Folies gourmandes', 'Martine Rancé', '184 chaussée de Tournai', 'Lille', '59000', 'France')
('GOURL', 'Gourmet Lanchonetes', 'André Fonseca', 'Av. Brasil 442', 'Campinas', '04876-786', 'Brazil')
('HANAR', 'Hanari Carnes', 'Mario Pontes', 'Rua do Paço 67', 'Rio de Janeiro', '05454-876', 'Brazil')
('LINOD', 'LINO-Delicateses', 'Felipe Izquierdo', 'Ave. 5 de Mayo Porlamar', 'I. de Margarita', '4980', 'Ven

In [151]:
run_query("SQL In", "SELECT * FROM Customers WHERE Country IN ('Germany', 'France', 'UK');")

run_query("SQL In / NOT IN", "SELECT * FROM Customers WHERE Country NOT IN ('Germany', 'France', 'UK');")

run_query("SQL In / IN (SELECT)", "SELECT * FROM Customers WHERE CustomerID IN (SELECT CustomerID FROM Orders);")

run_query("SQL In / NOT IN (SELECT)", "SELECT * FROM Customers WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders);")



 SQL In
SQL : SELECT * FROM Customers WHERE Country IN ('Germany', 'France', 'UK');
('AROUT', 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
('BLAUS', 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
('BLONP', 'Blondesddsl père et fils', 'Frédérique Citeaux', '24 place Kléber', 'Strasbourg', '67000', 'France')
('BONAP', "Bon app'", 'Laurence Lebihan', '12 rue des Bouchers', 'Marseille', '13008', 'France')
('BSBEV', "B's Beverages", 'Victoria Ashworth', 'Fauntleroy Circus', 'London', 'EC2 5NT', 'UK')
('CONSH', 'Consolidated Holdings', 'Elizabeth Brown', 'Berkeley Gardens 12  Brewery', 'London', 'WX1 6LT', 'UK')
('DRACD', 'Drachenblut Delikatessen', 'Sven Ottlieb', 'Walserweg 21', 'Aachen', '52066', 'Germany')
('DUMON', 'Du monde entier', 'Janine Labrune', '67 rue des Cinquante Otages', 'Nantes', '44000', 'France')
('EASTC', 'Eastern Connection', 'Ann Devon', '35 King George', 'London', 'WX3 6FW', 'UK')
('FOLIG'

In [158]:
run_query("SQL Between", "SELECT * FROM Products WHERE unitPrice BETWEEN 10 AND 20;")

run_query("SQL Between / NOT BETWEEN", "SELECT * FROM Products WHERE unitPrice NOT BETWEEN 10 AND 20;")

run_query("SQL Between / BETWEEN with IN", "SELECT * FROM Products WHERE unitPrice BETWEEN 10 AND 20 AND CategoryID IN (1,2,3);")

run_query("SQL Between / BETWEEN Text Values", "SELECT * FROM Products WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni' ORDER BY ProductName;")

run_query("SQL Between / NOT BETWEEN Text Values", "SELECT * FROM Products WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni' ORDER BY ProductName;")

run_query("SQL Between / BETWEEN Dates", "SELECT * FROM Orders WHERE OrderDate BETWEEN '1996-07-01' AND '1996-07-31';")



 SQL Between
SQL : SELECT * FROM Products WHERE unitPrice BETWEEN 10 AND 20;
(1, 'Chai', 1, 1, '10 boxes x 20 bags', 18.0, 39, 0, 10, 0)
(2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0, 17, 40, 25, 0)
(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0, 13, 70, 25, 0)
(15, 'Genen Shouyu', 6, 2, '24 - 250 ml bottles', 15.5, 39, 0, 5, 0)
(16, 'Pavlova', 7, 3, '32 - 500 g boxes', 17.45, 29, 0, 10, 0)
(21, "Sir Rodney's Scones", 8, 3, '24 pkgs. x 4 pieces', 10.0, 3, 40, 5, 0)
(25, 'NuNuCa Nuß-Nougat-Creme', 11, 3, '20 - 450 g glasses', 14.0, 76, 0, 30, 0)
(31, 'Gorgonzola Telino', 14, 4, '12 - 100 g pkgs', 12.5, 0, 70, 20, 0)
(34, 'Sasquatch Ale', 16, 1, '24 - 12 oz bottles', 14.0, 111, 0, 15, 0)
(35, 'Steeleye Stout', 16, 1, '24 - 12 oz bottles', 18.0, 20, 0, 15, 0)
(36, 'Inlagd Sill', 17, 8, '24 - 250 g  jars', 19.0, 112, 0, 20, 0)
(39, 'Chartreuse verte', 18, 1, '750 cc per bottle', 18.0, 69, 0, 5, 0)
(40, 'Boston Crab Meat', 19, 8, '24 - 4 oz tins', 18.4, 123, 0, 30, 0)
(42, 'Singapo

In [159]:
run_query("SQL Aliases", "SELECT CustomerID AS ID FROM Customers;")


 SQL Aliases
SQL : SELECT CustomerID AS ID FROM Customers;
(None,)
(None,)
(None,)
(None,)
(None,)
('ANATR',)
('ANTON',)
('AROUT',)
('BERGS',)
('BLAUS',)
('BLONP',)
('BOLID',)
('BONAP',)
('BOTTM',)
('BSBEV',)
('CACTU',)
('CENTC',)
('CHOPS',)
('COMMI',)
('CONSH',)
('DRACD',)
('DUMON',)
('EASTC',)
('ERNSH',)
('FAMIA',)
('FISSA',)
('FOLIG',)
('FOLKO',)
('FRANK',)
('FRANR',)
('FRANS',)
('FURIB',)
('GALED',)
('GODOS',)
('GOURL',)
('GREAL',)
('GROSR',)
('HANAR',)
('HILAA',)
('HUNGC',)
('HUNGO',)
('ISLAT',)
('KOENE',)
('LACOR',)
('LAMAI',)
('LAUGB',)
('LAZYK',)
('LEHMS',)
('LETSS',)
('LILAS',)
('LINOD',)
('LONEP',)
('MAGAA',)
('MAISD',)
('MEREP',)
('MORGK',)
('NORTS',)
('OCEAN',)
('OLDWO',)
('OTTIK',)
('PARIS',)
('PERIC',)
('PICCO',)
('PRINI',)
('QUEDE',)
('QUEEN',)
('QUICK',)
('RANCH',)
('RATTC',)
('REGGC',)
('RICAR',)
('RICSU',)
('ROMEY',)
('SANTG',)
('SAVEA',)
('SEVES',)
('SIMOB',)
('SPECD',)
('SPLIR',)
('SUPRD',)
('THEBI',)
('THECR',)
('TOMSP',)
('TORTU',)
('TRADH',)
('TRAIH',)
('VAFFE',

In [160]:
run_query("SQL Joins", "SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate FROM Orders INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;")


 SQL Joins
SQL : SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate FROM Orders INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;
(10248, 'Vins et alcools Chevalier', '1996-07-04 00:00:00.000')
(10249, 'Toms Spezialitäten', '1996-07-05 00:00:00.000')
(10250, 'Hanari Carnes', '1996-07-08 00:00:00.000')
(10251, 'Victuailles en stock', '1996-07-08 00:00:00.000')
(10252, 'Suprêmes délices', '1996-07-09 00:00:00.000')
(10253, 'Hanari Carnes', '1996-07-10 00:00:00.000')
(10254, 'Chop-suey Chinese', '1996-07-11 00:00:00.000')
(10255, 'Richter Supermarkt', '1996-07-12 00:00:00.000')
(10256, 'Wellington Importadora', '1996-07-15 00:00:00.000')
(10257, 'HILARION-Abastos', '1996-07-16 00:00:00.000')
(10258, 'Ernst Handel', '1996-07-17 00:00:00.000')
(10259, 'Centro comercial Moctezuma', '1996-07-18 00:00:00.000')
(10260, 'Ottilies Käseladen', '1996-07-19 00:00:00.000')
(10261, 'Que Delícia', '1996-07-19 00:00:00.000')
(10262, 'Rattlesnake Canyon Grocery', '1996-07-22 0

In [163]:
run_query("SQL Inner Join", "SELECT ProductID, ProductName, CategoryName FROM Products INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;")


 SQL Inner Join
SQL : SELECT ProductID, ProductName, CategoryName FROM Products INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;
(1, 'Chai', 'Beverages')
(2, 'Chang', 'Beverages')
(3, 'Aniseed Syrup', 'Condiments')
(4, "Chef Anton's Cajun Seasoning", 'Condiments')
(5, "Chef Anton's Gumbo Mix", 'Condiments')
(6, "Grandma's Boysenberry Spread", 'Condiments')
(7, "Uncle Bob's Organic Dried Pears", 'Produce')
(8, 'Northwoods Cranberry Sauce', 'Condiments')
(9, 'Mishi Kobe Niku', 'Meat/Poultry')
(10, 'Ikura', 'Seafood')
(11, 'Queso Cabrales', 'Dairy Products')
(12, 'Queso Manchego La Pastora', 'Dairy Products')
(13, 'Konbu', 'Seafood')
(14, 'Tofu', 'Produce')
(15, 'Genen Shouyu', 'Condiments')
(16, 'Pavlova', 'Confections')
(17, 'Alice Mutton', 'Meat/Poultry')
(18, 'Carnarvon Tigers', 'Seafood')
(19, 'Teatime Chocolate Biscuits', 'Confections')
(20, "Sir Rodney's Marmalade", 'Confections')
(21, "Sir Rodney's Scones", 'Confections')
(22, "Gustaf's Knäckebröd", 'Grains/C

In [165]:
run_query("SQL Left Join", "SELECT Customers.CustomerName, Orders.OrderID FROM Customers LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID ORDER BY Customers.CustomerName;")


 SQL Left Join
SQL : SELECT Customers.CustomerName, Orders.OrderID FROM Customers LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID ORDER BY Customers.CustomerName;
('Ana Trujillo Emparedados y helados', 10308)
('Ana Trujillo Emparedados y helados', 10625)
('Ana Trujillo Emparedados y helados', 10759)
('Ana Trujillo Emparedados y helados', 10926)
('Antonio Moreno Taquería', 10365)
('Antonio Moreno Taquería', 10507)
('Antonio Moreno Taquería', 10535)
('Antonio Moreno Taquería', 10573)
('Antonio Moreno Taquería', 10677)
('Antonio Moreno Taquería', 10682)
('Antonio Moreno Taquería', 10856)
('Around the Horn', 10355)
('Around the Horn', 10383)
('Around the Horn', 10453)
('Around the Horn', 10558)
('Around the Horn', 10707)
('Around the Horn', 10741)
('Around the Horn', 10743)
('Around the Horn', 10768)
('Around the Horn', 10793)
('Around the Horn', 10864)
('Around the Horn', 10920)
('Around the Horn', 10953)
('Around the Horn', 11016)
("B's Beverages", 10289)
("B's Beverages", 

In [171]:
run_query("SQL Right Join", "SELECT Orders.OrderID, Employees.LastName, Employees.FirstName FROM Orders RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID ORDER BY Orders.OrderID;")


 SQL Right Join
SQL : SELECT Orders.OrderID, Employees.LastName, Employees.FirstName FROM Orders RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID ORDER BY Orders.OrderID;
(10248, 'Buchanan', 'Steven')
(10249, 'Suyama', 'Michael')
(10250, 'Peacock', 'Margaret')
(10251, 'Leverling', 'Janet')
(10252, 'Peacock', 'Margaret')
(10253, 'Leverling', 'Janet')
(10254, 'Buchanan', 'Steven')
(10255, 'Dodsworth', 'Anne')
(10256, 'Leverling', 'Janet')
(10257, 'Peacock', 'Margaret')
(10258, 'Davolio', 'Nancy')
(10259, 'Peacock', 'Margaret')
(10260, 'Peacock', 'Margaret')
(10261, 'Peacock', 'Margaret')
(10262, 'Callahan', 'Laura')
(10263, 'Dodsworth', 'Anne')
(10264, 'Suyama', 'Michael')
(10265, 'Fuller', 'Andrew')
(10266, 'Leverling', 'Janet')
(10267, 'Peacock', 'Margaret')
(10268, 'Callahan', 'Laura')
(10269, 'Buchanan', 'Steven')
(10270, 'Davolio', 'Nancy')
(10271, 'Suyama', 'Michael')
(10272, 'Suyama', 'Michael')
(10273, 'Leverling', 'Janet')
(10274, 'Suyama', 'Michael')
(10275, 'D

In [172]:
run_query("SQL Full Join", "SELECT Customers.CustomerName, Orders.OrderID FROM Customers FULL OUTER JOIN Orders ON Customers.CustomerID=Orders.CustomerID ORDER BY Customers.CustomerName;")


 SQL Full Join
SQL : SELECT Customers.CustomerName, Orders.OrderID FROM Customers FULL OUTER JOIN Orders ON Customers.CustomerID=Orders.CustomerID ORDER BY Customers.CustomerName;
(None, 10643)
(None, 10692)
(None, 10702)
(None, 10835)
(None, 10952)
(None, 11011)
('Ana Trujillo Emparedados y helados', 10308)
('Ana Trujillo Emparedados y helados', 10625)
('Ana Trujillo Emparedados y helados', 10759)
('Ana Trujillo Emparedados y helados', 10926)
('Antonio Moreno Taquería', 10365)
('Antonio Moreno Taquería', 10507)
('Antonio Moreno Taquería', 10535)
('Antonio Moreno Taquería', 10573)
('Antonio Moreno Taquería', 10677)
('Antonio Moreno Taquería', 10682)
('Antonio Moreno Taquería', 10856)
('Around the Horn', 10355)
('Around the Horn', 10383)
('Around the Horn', 10453)
('Around the Horn', 10558)
('Around the Horn', 10707)
('Around the Horn', 10741)
('Around the Horn', 10743)
('Around the Horn', 10768)
('Around the Horn', 10793)
('Around the Horn', 10864)
('Around the Horn', 10920)
('Around 

In [173]:
run_query("SQL Self Join", "SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City FROM Customers A, Customers B WHERE A.CustomerID <> B.CustomerID AND A.City = B.City ORDER BY A.City;")


 SQL Self Join
SQL : SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City FROM Customers A, Customers B WHERE A.CustomerID <> B.CustomerID AND A.City = B.City ORDER BY A.City;
('Cactus Comidas para llevar', 'Océano Atlántico Ltda.', 'Buenos Aires')
('Cactus Comidas para llevar', 'Rancho grande', 'Buenos Aires')
('Océano Atlántico Ltda.', 'Cactus Comidas para llevar', 'Buenos Aires')
('Océano Atlántico Ltda.', 'Rancho grande', 'Buenos Aires')
('Rancho grande', 'Cactus Comidas para llevar', 'Buenos Aires')
('Rancho grande', 'Océano Atlántico Ltda.', 'Buenos Aires')
('Furia Bacalhau e Frutos do Mar', 'Princesa Isabel Vinhos', 'Lisboa')
('Princesa Isabel Vinhos', 'Furia Bacalhau e Frutos do Mar', 'Lisboa')
('Around the Horn', "B's Beverages", 'London')
('Around the Horn', 'Consolidated Holdings', 'London')
('Around the Horn', 'Eastern Connection', 'London')
('Around the Horn', 'North/South', 'London')
('Around the Horn', 'Seven Seas Imports', 'London')
("B's Bev

In [176]:
run_query("SQL Union", "SELECT City FROM Customers UNION SELECT City FROM Suppliers ORDER BY City;")


 SQL Union
SQL : SELECT City FROM Customers UNION SELECT City FROM Suppliers ORDER BY City;
(' 12Rio de Janeiro',)
('Aachen',)
('Albuquerque',)
('Anchorage',)
('Ann Arbor',)
('Annecy',)
('Barcelona',)
('Barquisimeto',)
('Bend',)
('Bergamo',)
('Berlin',)
('Bern',)
('Boise',)
('Boston',)
('Brandenburg',)
('Bruxelles',)
('Bräcke',)
('Buenos Aires',)
('Butte',)
('Campinas',)
('Caracas',)
('Charleroi',)
('Cork',)
('Cowes',)
('Cunewalde',)
('Cuxhaven',)
('Elgin',)
('Eugene',)
('Frankfurt',)
('Frankfurt a.M.',)
('Genève',)
('Graz',)
('Göteborg',)
('Helsinki',)
('I. de Margarita',)
('Kirkland',)
('Kobenhavn',)
('Köln',)
('Lander',)
('Lappeenranta',)
('Leipzig',)
('Lille',)
('Lisboa',)
('Liverpool',)
('London',)
('Luleå',)
('Lyngby',)
('Lyon',)
('Madrid',)
('Manchester',)
('Mannheim',)
('Marseille',)
('Melbourne',)
('Montceau',)
('Montréal',)
('México D.F.',)
('München',)
('Münster',)
('Nantes',)
('New Orleans',)
('Osaka',)
('Oulu',)
('Oviedo',)
('Paris',)
('Portland',)
('Ravenna',)
('Reggio E

In [177]:
run_query("SQL Group By", "SELECT COUNT(CustomerID), Country FROM Customers GROUP BY Country;")


 SQL Group By
SQL : SELECT COUNT(CustomerID), Country FROM Customers GROUP BY Country;
(3, 'Argentina')
(2, 'Austria')
(2, 'Belgium')
(9, 'Brazil')
(3, 'Canada')
(2, 'Denmark')
(2, 'Finland')
(11, 'France')
(10, 'Germany')
(1, 'Ireland')
(3, 'Italy')
(5, 'Mexico')
(1, 'Norway')
(1, 'Poland')
(2, 'Portugal')
(5, 'Spain')
(2, 'Sweden')
(2, 'Switzerland')
(7, 'UK')
(13, 'USA')
(4, 'Venezuela')


In [178]:
run_query("SQL Having", "SELECT COUNT(CustomerID), Country FROM Customers GROUP BY Country HAVING COUNT(CustomerID) > 5;")


 SQL Having
SQL : SELECT COUNT(CustomerID), Country FROM Customers GROUP BY Country HAVING COUNT(CustomerID) > 5;
(9, 'Brazil')
(11, 'France')
(10, 'Germany')
(7, 'UK')
(13, 'USA')


In [181]:
run_query("SQL Exists", "SELECT CompanyName FROM Suppliers WHERE EXISTS (SELECT ProductName FROM Products WHERE Products.SupplierID = Suppliers.supplierID AND unitPrice < 20);")


 SQL Exists
SQL : SELECT CompanyName FROM Suppliers WHERE EXISTS (SELECT ProductName FROM Products WHERE Products.SupplierID = Suppliers.supplierID AND unitPrice < 20);
('Exotic Liquids',)
('New Orleans Cajun Delights',)
('Tokyo Traders',)
("Mayumi's",)
('Pavlova Ltd.',)
('Specialty Biscuits Ltd.',)
('PB Knäckebröd AB',)
('Refrescos Americanas LTDA',)
('Heli Süßwaren GmbH & Co. KG',)
('Plutzer Lebensmittelgroßmärkte AG',)
('Formaggi Fortini s.r.l.',)
('Norske Meierier',)
('Bigfoot Breweries',)
('Svensk Sjöföda AB',)
('Aux joyeux ecclésiastiques',)
('New England Seafood Cannery',)
('Leka Trading',)
('Lyngbysild',)
('Zaanse Snoepfabriek',)
('Karkki Oy',)
("G'day Mate",)
('Ma Maison',)
('Pasta Buttini s.r.l.',)
('Escargots Nouveaux',)


In [186]:
# SQLite에서 하위 쿼리 결과와 비교할 때는 ANY나 ALL 대신 'IN', 'NOT IN'을 사용한다. >> 더 표준적이고 호환성이 높은 방법
run_query("SQL Any,All", "SELECT ProductName FROM Products WHERE ProductID IN (SELECT ProductID FROM Order_details WHERE Quantity = 10);")


 SQL Any,All
SQL : SELECT ProductName FROM Products WHERE ProductID IN (SELECT ProductID FROM Order_details WHERE Quantity = 10);
('Chai',)
('Chang',)
("Chef Anton's Cajun Seasoning",)
("Uncle Bob's Organic Dried Pears",)
('Northwoods Cranberry Sauce',)
('Ikura',)
('Queso Cabrales',)
('Konbu',)
('Tofu',)
('Genen Shouyu',)
('Pavlova',)
('Alice Mutton',)
('Carnarvon Tigers',)
('Teatime Chocolate Biscuits',)
("Sir Rodney's Scones",)
('Tunnbröd',)
('Guaraná Fantástica',)
('NuNuCa Nuß-Nougat-Creme',)
('Gumbär Gummibärchen',)
('Schoggi Schokolade',)
('Rössle Sauerkraut',)
('Thüringer Rostbratwurst',)
('Nord-Ost Matjeshering',)
('Gorgonzola Telino',)
('Mascarpone Fabioli',)
('Geitost',)
('Sasquatch Ale',)
('Steeleye Stout',)
('Gravad lax',)
('Côte de Blaye',)
('Chartreuse verte',)
('Boston Crab Meat',)
("Jack's New England Clam Chowder",)
('Singaporean Hokkien Fried Mee',)
('Ipoh Coffee',)
('Gula Malacca',)
('Spegesild',)
('Zaanse koeken',)
('Maxilaku',)
('Manjimup Dried Apples',)
('Filo Mix

In [193]:
# SQLite에서는 SELECT INTO 문법을 지원하지 않는다.
# run_query("SQL Select Into", "SELECT * INTO CustomersBackup2017 FROM Customers;")

In [201]:
run_query("SQL Insert Into Select", "INSERT INTO Customers (CustomerName, City, Country) SELECT CompanyName, City, Country FROM Suppliers;")


 SQL Insert Into Select
SQL : INSERT INTO Customers (CustomerName, City, Country) SELECT CompanyName, City, Country FROM Suppliers;


In [202]:
run_query("SQL Case", "SELECT CustomerName, City, Country FROM Customers ORDER BY (CASE WHEN City IS NULL THEN Country ELSE City END);")


 SQL Case
SQL : SELECT CustomerName, City, Country FROM Customers ORDER BY (CASE WHEN City IS NULL THEN Country ELSE City END);
('Que Delícia', ' 12Rio de Janeiro', 'Brazil')
('Drachenblut Delikatessen', 'Aachen', 'Germany')
('Rattlesnake Canyon Grocery', 'Albuquerque', 'USA')
('Old World Delicatessen', 'Anchorage', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
('Regina Murphy', 'Ann Arbor', 'USA')
('Sales Representative', 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
("Grandma Kelly's Homestead", 'Ann Arbor', 'USA')
('Gai pâturage', 'Annecy', 'France')
('Gai pâturage', 'Annecy', 'France')
('Eliane Noz', 'Annecy', 'France')
('Sales Representative', 'Annecy', 'France')
('Gai pâturage', 'Annecy', 'France')
('Gai pâturage', 'Annecy', 'France')
('Gai pâturag

In [203]:
run_query("SQL Null Functions", "SELECT ProductName, UnitPrice * (UnitsInStock + UnitsOnOrder) FROM Products;")


 SQL Null Functions
SQL : SELECT ProductName, UnitPrice * (UnitsInStock + UnitsOnOrder) FROM Products;
('Chai', 702.0)
('Chang', 1083.0)
('Aniseed Syrup', 830.0)
("Chef Anton's Cajun Seasoning", 1166.0)
("Chef Anton's Gumbo Mix", 0.0)
("Grandma's Boysenberry Spread", 3000.0)
("Uncle Bob's Organic Dried Pears", 450.0)
('Northwoods Cranberry Sauce', 240.0)
('Mishi Kobe Niku', 2813.0)
('Ikura', 961.0)
('Queso Cabrales', 1092.0)
('Queso Manchego La Pastora', 3268.0)
('Konbu', 144.0)
('Tofu', 813.75)
('Genen Shouyu', 604.5)
('Pavlova', 506.04999999999995)
('Alice Mutton', 0.0)
('Carnarvon Tigers', 2625.0)
('Teatime Chocolate Biscuits', 229.99999999999997)
("Sir Rodney's Marmalade", 3240.0)
("Sir Rodney's Scones", 430.0)
("Gustaf's Knäckebröd", 2184.0)
('Tunnbröd', 549.0)
('Guaraná Fantástica', 90.0)
('NuNuCa Nuß-Nougat-Creme', 1064.0)
('Gumbär Gummibärchen', 468.45)
('Schoggi Schokolade', 2151.1)
('Rössle Sauerkraut', 1185.6000000000001)
('Thüringer Rostbratwurst', 0.0)
('Nord-Ost Matjeshe

In [206]:
# SQLite에서 CREATE PROCEDURE, GO 문법을 모두 지원하지 않는다.
# run_query("SQL Stored Procedures", "CREATE PROCEDURE SelectAllCustomers AS SELECT * FROM Customers GO;")

In [205]:
run_query("SQL Comments", "-- Select all: SELECT * FROM Customers;")


 SQL Comments
SQL : -- Select all: SELECT * FROM Customers;
